In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import json
file = open('data.json', 'r', encoding='utf-8')
data = json.load(file)

import koreanize_matplotlib

In [3]:
# Parse personal data
combined_rows = []

# Match entries by totalScore and grade
for stat_id, stat_data in data["statistics"].items():
    for stu_id, stu_data in data["students"].items():
        if (stat_data["grade"] == stu_data["studentGrade"] and
            stat_data["totalScore"] == stu_data["totalScore"] and
                stat_data["scores"] == stu_data["scores"]):

            row = {
                "ID_stat": stat_id,
                "ID_student": stu_id,
                "Gender": stat_data.get("gender"),
                "Grade": stat_data.get("grade"),
                "SurveyTime": stat_data.get("date"),
                "ParsedTime": stu_data.get("parsedDate"),
                "TotalScore": stat_data.get("totalScore")
            }
            # Include category scores
            row.update(stat_data["scores"])
            combined_rows.append(row)
            break

# Convert to DataFrame
df_combined = pd.DataFrame(combined_rows)

# Print final table
print(df_combined.to_string(index=False))

             ID_stat           ID_student Gender Grade SurveyTime         ParsedTime  TotalScore  불안 및 우울 문제  심리외상 문제  외현화 문제  자살 및 위기 문제  학교생활적응 문제
-ORO3afgAtqlPL9nX2G5 -ORO3afb_TN98Kofxvo2   girl   2학년  5월 29일 7시  2025-5-29 7:45:26          80          24       14       9          18         15
-ORO42E6AZfcaferUZKV -ORO42E2e1pii41jNKbl   girl   1학년  5월 29일 7시  2025-5-29 7:47:23           0           0        0       0           0          0
-ORO47TnvMiiv0gHk-3i -ORO47TjdZ_5jV43PcXi    boy   2학년  5월 29일 7시  2025-5-29 7:47:45           4           0        0       2           0          2
-ORO4eKQsY3AF-Cji_Rk -ORO4eKMBcwY5NWfbf_9   girl   2학년  5월 29일 7시   2025-5-29 7:50:3           7           6        0       0           0          1
-ORO5pZyLjX3rRv7QpeL -ORO5pZuPnJomqtF8JFn    boy   1학년  5월 29일 7시  2025-5-29 7:55:12          19           9        4       3           0          3
-ORO5zoeU-yf5-uvZosn -ORO5zobt7VtPF24uHBx    boy   1학년  5월 29일 7시  2025-5-29 7:55:54           9          

In [4]:
import pandas as pd

question_records = []

# Iterate through each student's stat data
for stat_id, stat_data in data["statistics"].items():
    gender = stat_data["gender"]
    grade = stat_data["grade"]
    total_score = stat_data["totalScore"]
    questions = stat_data.get("question", [])

    for q in questions:
        if q:
            question_records.append({
                "student_id": stat_id,
                "Gender": gender,
                "Grade": grade,
                "Question": q["question"],
                "Answer": q["answer"],
                "Type": q["type"],
                "TotalScore": total_score
            })

# Convert to DataFrame
question_df = pd.DataFrame(question_records)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 1: Encode questions and answers
question_encoder = LabelEncoder()
question_df["QuestionEncoded"] = question_encoder.fit_transform(
    question_df["Question"])

# Create a sequence of (question, answer) pairs for each student
sequences = []
targets = []

for student_id, group in question_df.groupby("student_id"):
    group = group.reset_index(drop=True)
    for i in range(1, len(group)):
        # Previous (question, answer)
        prev_question = group.loc[i - 1, "QuestionEncoded"]
        prev_answer = group.loc[i - 1, "Answer"]
        # Target is current answer
        target_answer = group.loc[i, "Answer"]
        sequences.append([prev_question, prev_answer])
        targets.append(target_answer)

X = np.array(sequences)
y = np.array(targets)

# Step 2: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Step 3: Train logistic regression model
clf = LogisticRegression(max_iter=200)
clf.fit(X_train, y_train)

# Step 4: Predict and evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.6852300242130751

In [14]:
def predict_next_answer(sequence, model, encoder):
    """
    Predict the next answer given a sequence of (Question, Answer) pairs.

    Parameters:
    - sequence: list of tuples [(question_str, answer), ...]
    - model: trained sklearn classifier
    - encoder: fitted LabelEncoder for question encoding

    Returns:
    - predicted next answer
    """
    if not sequence:
        raise ValueError("Input sequence is empty.")

    last_question, last_answer = sequence[-1]

    # Encode the last question
    try:
        last_question_encoded = encoder.transform([last_question])[0]
    except ValueError:
        raise ValueError(f"Question '{last_question}' not in encoder classes.")

    # Prepare input feature
    input_features = np.array([[last_question_encoded, last_answer]])

    # Predict next answer
    return model.predict(input_features)[0]

  # Example: last 3 answered questions by a student
input_sequence = [
    ("사람과 있을 때 긴장을 많이 한다.", 1),
    ("이유 없이 감정기복이 심하다.", 2),
    ("심각하게 죄책감을 느낀다.", 1)
]

predicted_answer = predict_next_answer(input_sequence, clf, question_encoder)
print(f"Predicted next answer: {predicted_answer}")

Predicted next answer: 0
